In [6]:
import pandas as pd
import re
import os
import json
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata
from collections import Counter, defaultdict

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm
from string import punctuation

from pandas_datareader import data as pdr 
import yfinance as yf 

yf.pdr_override()

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

"""
this is where different from version 1
"""
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Universidad/TESIS/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Universidad/TESIS/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Dirección de la data:
'./Scrape_data/Scrape_data/10_k/[cik]/grabbed_text/[cik]_[date]'

## Creación de diccionario de palabras

In [7]:
ticker_library = pd.read_csv('tickers.csv')

ticker_selected = pd.read_csv('SP500_component_stocks.csv',header = None)
ticker_selected.columns = ['name','ticker']

display(ticker_selected)


/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,ticker
0,Agilent Technologies Inc.,A
1,American Airlines Group Inc.,AAL
2,Advance Auto Parts Inc.,AAP
3,Apple Inc.,AAPL
4,AbbVie Inc.,ABBV
...,...,...
500,Xylem Inc.,XYL
501,Yum! Brands Inc.,YUM
502,Zimmer Biomet Holdings Inc.,ZBH
503,Zions Bancorporation N.A.,ZION


In [8]:
ticker_cik_df = pd.DataFrame()

ticker_list = ticker_selected.ticker.unique()

cik_list = []

for ticker in ticker_list:    
    try:
        cik_list.append(list(ticker_library[ticker_library.ticker == ticker].secfilings)[0][-10:])
        
    except:
        cik_list.append('')

ticker_cik_df['cik'] = cik_list
ticker_cik_df['ticker'] = ticker_list

ticker_cik_df = ticker_cik_df[ticker_cik_df['cik'] != '']

ticker_cik_df.head()


,cik,ticker
0,0001090872,A
1,0000006201,AAL
2,0001158449,AAP
3,0000320193,AAPL
4,0001551152,ABBV


In [9]:
#use a sample to analysis the overall program
# sample1 = ticker_cik_df.iloc[0:50]
# sample2 = ticker_cik_df.iloc[50:100]
# sample3 = ticker_cik_df.iloc[100:150]
# sample4 = ticker_cik_df.iloc[150:200]
# sample5 = ticker_cik_df.iloc[200:250]
# sample6 = ticker_cik_df.iloc[250:300]
# sample7 = ticker_cik_df.iloc[300:350]
# sample8 = ticker_cik_df.iloc[350:400]
# sample9 = ticker_cik_df.iloc[400:450]
# sample10 = ticker_cik_df.iloc[450:500]
ticker_cik_sample = ['0000049826','0000816284']


In [10]:
all_data = {}

for cik, ticker in zip(cik_list, ticker_list):
    
    all_data[ticker] = {}

    all_data[ticker]['cik'] = cik
    all_data[ticker]['10ks'] = {}
    all_data[ticker]['10qs'] = {}


In [12]:
pathname_10k = './Scrape_data/10_k/'
pathname_10q = './Scrape_data/10_q/'

In [13]:

def Get_Ex_Ret(ticker,fillingdate):
    
    start = datetime.strptime(fillingdate, '%Y-%m-%d')
    
    """
    change from 10 to 120 to ensure engough days
    """
    end = start + timedelta(120)
    
    price_120_days = pdr.get_data_yahoo(ticker, start, end)
    
    """
    change from 5 days to 63 days---which is the length of three month
    """
    ret_4d = price_120_days.iloc[62]['Adj Close'] / price_120_days.iloc[0]['Adj Close'] - 1
    return 100 * ret_4d

In [14]:
def remove_punct(string):
    return re.sub(r"[{}]+".format(punctuation), "", string)

In [15]:
def filter_words(string):
    return bool(re.match(r'^[a-z\']+$', string))


In [16]:

for ticker in all_data.keys():
    
    cik = all_data[ticker]['cik']
    
    try:
        files_10k = os.listdir(pathname_10k + cik + '/grabbed_text/')
    except:
        break
    
    for file_10k in files_10k:
        
        release_10k = file_10k[-14:-4]
        
        """
        adding beta = 1, denoting the market return value at that time to calculate excess return.
        """
        
        try:
            excess_return = Get_Ex_Ret(ticker, release_10k)
            """
            adding a market return term and subtracting it(assuming beta = 1)
            """
            market_return = Get_Ex_Ret('SPY', release_10k)
            excess_return = excess_return - market_return
            
        except:
            break
    
        with open(pathname_10k + cik + '/grabbed_text/' + file_10k, encoding = "utf8") as f:
            string_temp = f.read().lower()
        
        filtered_words = [remove_punct(word) for word in string_temp.split() if word not in stop_words]
        
        filtered_words = list(filter(filter_words, filtered_words))
        
        filtered_words = [lemmatizer.lemmatize(word, pos = 'v') for word in filtered_words]
        filtered_words = [lemmatizer.lemmatize(word, pos = 'n') for word in filtered_words]
        
        
        if filtered_words != []:
            all_data[ticker]['10ks'][release_10k] = {}
            all_data[ticker]['10ks'][release_10k]['ex_return'] = excess_return
            all_data[ticker]['10ks'][release_10k]['words'] = ' '.join(filtered_words)


In [17]:
"""
This is where different from version 1, adding a beta term to calculate excess return.
"""

for ticker in all_data.keys():
    
    cik = all_data[ticker]['cik']
    
    try:
        files_10q = os.listdir(pathname_10q + cik + '/grabbed_text/')
    except:
        break
    
    for file_10q in files_10q:
        
        release_10q = file_10q[-14:-4]       
        
        
        try:
            excess_return = Get_Ex_Ret(ticker, release_10q)
            """
            adding a market return term and subtracting it(assuming beta = 1)
            """
            market_return = Get_Ex_Ret('SPY', release_10k)
            excess_return = excess_return - market_return
        
        except:
            break
    
        with open(pathname_10q + cik + '/grabbed_text/' + file_10q, encoding = "utf8") as f:
            string_temp = f.read().lower()
        
        filtered_words = [remove_punct(word) for word in string_temp.split() if word not in stop_words]
        
        filtered_words = list(filter(filter_words, filtered_words))
        
        filtered_words = [lemmatizer.lemmatize(word, pos = 'v') for word in filtered_words]
        filtered_words = [lemmatizer.lemmatize(word, pos = 'n') for word in filtered_words]
        
        
        if filtered_words != []:
            all_data[ticker]['10qs'][release_10q] = {}
            all_data[ticker]['10qs'][release_10q]['ex_return'] = excess_return
            all_data[ticker]['10qs'][release_10q]['words'] = ' '.join(filtered_words)


### Estructura de datos

## Analisis de 10K con tfidf y bag-of-words

In [19]:
"""
This part is to compute the tf values for the words in 10ks and collect the overall word list for computing idf in the next step
"""
document_num_10k = 0

word_list_10k = defaultdict(int)


for ticker in all_data:
    
    for date in all_data[ticker]['10ks']:
        
        document_num_10k += 1
        
        all_data[ticker]['10ks'][date]['tf'] = Counter(all_data[ticker]['10ks'][date]['words'].split())
        
        for word in all_data[ticker]['10ks'][date]['tf']:
            
            word_list_10k[word] += 1
        

In [20]:
idf_10k = {}

for word in word_list_10k:
    
    idf_10k[word] = np.log(document_num_10k / (1 + word_list_10k[word]))

In [21]:
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

pair_list_10k = defaultdict(int)

for ticker in all_data:
    
    for date in all_data[ticker]['10ks']:
        
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
        
        fitted = vectorizer.fit_transform([all_data[ticker]['10ks'][date]['words']])
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
        all_data[ticker]['10ks'][date]['tf_pair'] = df_temp.to_dict()[0]
        for pair in all_data[ticker]['10ks'][date]['tf_pair']:
            
            pair_list_10k[pair] += 1
        

In [22]:
pair_list_10k

defaultdict(int, {})

In [23]:
idf_10k_pair = {}

for pair in pair_list_10k:
    
    idf_10k_pair[pair] = np.log(document_num_10k / (1 + pair_list_10k[pair]))

In [24]:
triple_list_10k = defaultdict(int)


for ticker in all_data:
    
    for date in all_data[ticker]['10ks']:
        
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (3, 3))
        
        fitted = vectorizer.fit_transform([all_data[ticker]['10ks'][date]['words']])
        
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
        
        all_data[ticker]['10ks'][date]['tf_triple'] = df_temp.to_dict()[0]
        
        for triple in all_data[ticker]['10ks'][date]['tf_triple']:
            
            triple_list_10k[triple] += 1
            
idf_10k_triple = {}

for triple in triple_list_10k:
    
    idf_10k_triple[triple] = np.log(document_num_10k / (1 + triple_list_10k[triple]))

### doing the same to 10qs

In [25]:
#Misma ejecución para 10 Q

document_num_10q = 0

word_list_10q = defaultdict(int)


for ticker in all_data:
    
    for date in all_data[ticker]['10qs']:
        
        document_num_10q += 1
        
        all_data[ticker]['10qs'][date]['tf'] = Counter(all_data[ticker]['10qs'][date]['words'].split())
        
        for word in all_data[ticker]['10qs'][date]['tf']:
            
            word_list_10q[word] += 1
               

In [26]:
idf_10q = {}

for word in word_list_10q:
    
    idf_10q[word] = np.log(document_num_10q / (1 + word_list_10q[word]))
    

In [27]:
pair_list_10q = defaultdict(int)

for ticker in all_data:
    
    for date in all_data[ticker]['10qs']:
        
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
        
        fitted = vectorizer.fit_transform([all_data[ticker]['10qs'][date]['words']])
        
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
        
        all_data[ticker]['10qs'][date]['tf_pair'] = df_temp.to_dict()[0]
        
        for pair in all_data[ticker]['10qs'][date]['tf_pair']:
            
            pair_list_10q[pair] += 1

In [28]:
idf_10q_pair = {}

for pair in pair_list_10q:
    
    idf_10q_pair[pair] = np.log(document_num_10q / (1 + pair_list_10q[pair]))

In [29]:

triple_list_10q = defaultdict(int)


for ticker in all_data:
    
    for date in all_data[ticker]['10qs']:
        
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (3, 3))
        
        fitted = vectorizer.fit_transform([all_data[ticker]['10qs'][date]['words']])
        
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
        
        all_data[ticker]['10qs'][date]['tf_triple'] = df_temp.to_dict()[0]
        
        for triple in all_data[ticker]['10qs'][date]['tf_triple']:
            
            triple_list_10q[triple] += 1
            
idf_10q_triple = {}

for triple in triple_list_10q:
    
    idf_10q_triple[triple] = np.log(document_num_10q / (1 + triple_list_10q[triple]))

### Estructura de datos

In [30]:
idf_10k

{}

In [31]:
idf_10q_pair

{}

In [32]:
idf_10q_triple

{}

## Store the data for future use 

In [33]:
for ticker in all_data:
    
    for date in all_data[ticker]['10qs']:
        
        del all_data[ticker]['10qs'][date]['words']
    
    for date in all_data[ticker]['10ks']:
        
        del all_data[ticker]['10ks'][date]['words']


In [34]:
with open('all_data.json', 'w') as json_file:
    json_file.write(json.dumps(all_data))

with open('idf_10k.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10k))

with open('idf_10q.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10q))
    

In [35]:
with open('idf_10k_pair.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10k_pair))

with open('idf_10q_pair.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10q_pair))
    
with open('idf_10k_triple.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10k_triple))

with open('idf_10q_triple.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10q_triple))    
